In [125]:
from sklearn.feature_extraction import DictVectorizer#转换工具，将list转换成为一个数组
from sklearn import preprocessing
from sklearn import tree #创建决策树
import numpy as np

In [126]:
# dataset: https://blog.csdn.net/leafage_m/article/details/79560791

    dataSet = [
        # 1
        ['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', 0.697, 0.460, '好瓜'],
        # 2
        ['乌黑', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', 0.774, 0.376, '好瓜'],
        # 3
        ['乌黑', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', 0.634, 0.264, '好瓜'],
        # 4
        ['青绿', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', 0.608, 0.318, '好瓜'],
        # 5
        ['浅白', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', 0.556, 0.215, '好瓜'],
        # 6
        ['青绿', '稍蜷', '浊响', '清晰', '稍凹', '软粘', 0.403, 0.237, '好瓜'],
        # 7
        ['乌黑', '稍蜷', '浊响', '稍糊', '稍凹', '软粘', 0.481, 0.149, '好瓜'],
        # 8
        ['乌黑', '稍蜷', '浊响', '清晰', '稍凹', '硬滑', 0.437, 0.211, '好瓜'],

        # ----------------------------------------------------
        # 9
        ['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '硬滑', 0.666, 0.091, '坏瓜'],
        # 10
        ['青绿', '硬挺', '清脆', '清晰', '平坦', '软粘', 0.243, 0.267, '坏瓜'],
        # 11
        ['浅白', '硬挺', '清脆', '模糊', '平坦', '硬滑', 0.245, 0.057, '坏瓜'],
        # 12
        ['浅白', '蜷缩', '浊响', '模糊', '平坦', '软粘', 0.343, 0.099, '坏瓜'],
        # 13
        ['青绿', '稍蜷', '浊响', '稍糊', '凹陷', '硬滑', 0.639, 0.161, '坏瓜'],
        # 14
        ['浅白', '稍蜷', '沉闷', '稍糊', '凹陷', '硬滑', 0.657, 0.198, '坏瓜'],
        # 15
        ['乌黑', '稍蜷', '浊响', '清晰', '稍凹', '软粘', 0.360, 0.370, '坏瓜'],
        # 16
        ['浅白', '蜷缩', '浊响', '模糊', '平坦', '硬滑', 0.593, 0.042, '坏瓜'],
        # 17
        ['青绿', '蜷缩', '沉闷', '稍糊', '稍凹', '硬滑', 0.719, 0.103, '坏瓜']
    ]

    # 特征值列表
    labels = ['色泽', '根蒂', '敲击', '纹理', '脐部', '触感', '密度', '含糖率']


In [127]:
labelList = [] # 好瓜or坏瓜

for row in dataSet:
    labelList.append(row[-1])
lb = preprocessing.LabelBinarizer()
Y = lb.fit_transform(labelList)
print(Y)

[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


In [128]:
print(Y[1])
print(Y[1][0])

[1]
1


In [129]:
# 计算信息熵
# data: [1,2,4,5] 样本的序号
def computeEnt(data):
    num = 0
    for i in data:
        num += Y[i]
    p = num/len(data)
    if p == 0 or p == 1:
        return 0
    return -(p*np.log2(p)+(1-p)*np.log2(1-p))[0]

In [130]:
# 计算信息增益
# 在特征值是连续值时，如何减少计算量 http://blog.sina.com.cn/s/blog_68ffc7a40100urn3.html
# data: [1,2,4,5] 样本的序号
# attr: 类型:int，labels[attr]表示在此属性下计算信息增益
# return: D, gain, divideData  
# D:表示样本在该属性下取值的数量  [2,5,1]: 表示在该属性下共分为三类（离散属性值必为两类），每一类中分别由2个，5个，1个样本
# gain： 信息增益
# divideData: 只有离散值属性时返回，连续值返回为None， 返回值表示该属性下最优划分值
def computeGainC45(data, attr):    
    if(attr not in [6, 7]):# 离散值属性
        ent = computeEnt(data)
        group = {}
        for i in data:
            if dataSet[i][attr] not in group:
                group[dataSet[i][attr]] = [i]
            else:
                group[dataSet[i][attr]].append(i)
#     print(group) # {'沉闷': [1, 8], '浊响': [5, 6, 11, 15], '清脆': [10]}
        D = []
        for key in group:
            D.append(len(group[key]))
            ent -= len(group[key])/len(data)*computeEnt(group[key])
        gain = ent
        return D, gain, None
#     print(ent)


    else: # 连续值属性
        tmp = [] # 临时数组
        # 生成[[0,1],[1,3],[4,5]]的序列，每一个list第一个为该样本在此属性的取值，第二个元素为样本序号（用于判断其类别），最后按照属性取值顺序排列
        for i in range(len(data)):
            tmp.append([dataSet[data[i]][attr], data[i]])
        tmp.sort() # 排序
        gain = 0 
        divideData = 0 # 储存候选划分值，以便找到最优划分值，然后再计算该属性的信息增益率
        for i in range(len(data) - 1):
            if(Y[tmp[i][1]] != Y[tmp[i+1][1]]):
                divide = (tmp[i][0] + tmp[i+1][0])/2
                p1, p2, n1, n2 = 0, 0, 0, 0 # 记录正例反例的数量用以计算信息增益，下标1表示在该属性下取值小于divideData, 下标2表示大于divideData
                for d in data:
                    if(dataSet[d][attr] < divide and Y[d] == [1]):
                        p1 += 1
                    elif(dataSet[d][attr] < divide and Y[d] == [0]):
                        n1 += 1
                    elif(dataSet[d][attr] > divide and Y[d] == [1]):
                        p2 += 1
                    elif(dataSet[d][attr] > divide and Y[d] == [0]):
                        n2 += 1
                D = [p1+n1, p2+n2]
                if(n1 == 0 or p1 == 0):
                    ent1 = 0
                else:
                    ent1 = -(p1+n1)/len(data)*(p1/(p1+n1)*np.log2(p1/(p1+n1))+n1/(p1+n1)*np.log2(n1/(p1+n1)))
                if(n2 == 0 or p2 == 0):
                    ent2 = 0
                else:
                    ent2 = -(p2+n2)/len(data)*(p2/(p2+n2)*np.log2(p2/(p2+n2))+n2/(p2+n2)*np.log2(n2/(p2+n2))) 
                ent = computeEnt(data) - ent1 - ent2
                if(ent>gain):
                    gain = ent
                    divideData = divide
        return D, gain, divideData

# for i in range(8):
#     print(computeGainC45([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],i))

In [131]:
# 计算增益率
# gain:信息增益
# data, attr同上
def computeGainRatio(D, gain):
    IV = 0 # intrinsic value
    for i in D:
        IV -= i/sum(D)*np.log2(i/sum(D))
    gainRatio = gain/IV    
    return gainRatio

### C4.5对取值数目较少的属性有偏好，所以我们并不直接选GainRatio最大的属性值，
### 而是先从候选划分属性中找出Gain高于平均水平的属性，然后再从中选GainRatio最大的属性值。
意思是先计算高水平的属性，再在这些属性里计算增益率
可以避免gain ratio公式中 分母接近于0或者等于0 （若某属性下的样本全为同一类别，那么分母计算为0）

In [147]:
# 找到最优划分属性
# data 同上
# attrList: [0,1,2,3,4] 代表候选属性
# return gainList[0]: [gain, gainRatio, divideData, i]
def chooseBestAttrC45(data, attrList):
    gainList = [] # 记录每个属性的gain,gainRatio,divideData以及属性序号
    gainSum = 0
    for i in range(len(attrList)):
        attr = attrList[i]
        D, gain, divideData = computeGainC45(data, attr)
        gainList.append([D, gain, divideData, i])
        gainSum += gain
    gainAvg = gainSum/len(attrList) # gain的平均水平
    #先从候选划分属性中找出Gain高于平均水平的属性，然后再从中选GainRatio最大的属性值
    for i in range(len(gainList)-1, -1, -1): # 倒序删除不会出错
        if(gainList[i][1] < gainAvg):
            del(gainList[i])
    for i in range(len(gainList)):
        gainRatio = computeGainRatio(gainList[i][0], gainList[i][1])
        gainList[i][0] = gainList[i][1]
        gainList[i][1] = gainRatio
        
    gainList = sorted(gainList, key = lambda x:x[1], reverse = True)
    print(gainList)
    return gainList[0]
# chooseBestAttrC45([i for i in range(17)],[i for i in range(8)])

In [148]:
labels

['色泽', '根蒂', '敲击', '纹理', '脐部', '触感', '密度', '含糖率']

In [149]:
# 选择最佳划分属性
# data 同上
# attrList: [0,1,2,3,4] 代表候选属性
def generateTreeC45(data, attrList):
    num = 0
    for i in data:
        num += Y[i][0]
    if num == 0 or num == len(data):# 如果D中样本全为同一类别C，返回
        return {'label':Y[data[0]][0], 'sampleNum':num if num != 0 else len(data)} # 标记为C类
    
    if attrList == []: # A空了却依然不能完整分类  （少了一个D中样本在A上取值相同的判断）
        pos = num # 正例
        neg = len(data) - num # 反例
        return {'label':1,'sampleNum':[pos,neg]} if pos > neg else {'label':0,'sampleNum':[pos,neg]}
        
        
        # 标记为D中分类最多的
    
    best = chooseBestAttrC45(data, attrList) # [gain, gainRatio, divideData, i]
    C45Tree = {labels[best[3]]:{}} # 找到最优划分属性，对属性的每一个取值都再次寻找决策树
#     print(labels[attrBest])
    group = {}
    if(best[2] == None): # 离散型属性
        del(attrList[best[3]]) # 离散值属性则在子树中删除该属性，但是连续值属性不需要删除
        for i in data: # 对data分割成： {'沉闷': [1, 8], '浊响': [5, 6, 11, 15], '清脆': [10]}
            if dataSet[i][best[3]] not in group:
                group[dataSet[i][best[3]]] = [i]
            else:
                group[dataSet[i][best[3]]].append(i)
    else: # 连续型属性
        group['小于'+str(best[2])] = []
        group['大于'+str(best[2])] = []
        for i in data:
            if dataSet[i][best[3]] <= best[2]:
                group['小于'+str(best[2])].append(i)
            else:
                group['大于'+str(best[2])].append(i)
    
    
    for key in group:
        C45Tree[labels[best[3]]][key] = generateTreeC45(group[key], attrList)
    
      
    
    return C45Tree
        

In [150]:
C45 = generateTreeC45([i for i in range(17)], [i for i in range(8)])

[[0.2624392604045632, 0.8131172796431959, 0.3815, 6], [0.34929372233065203, 0.6684287745104068, 0.126, 7], [0.3805918973682686, 0.2630853587192754, None, 3], [0.289158782841679, 0.18672689918448795, None, 4]]
[[0.2751070248348759, 0.7031605128092757, 0.5745, 6], [0.6612262562697893, 0.6640634993895833, 0.20450000000000002, 7], [0.6835719528431211, 0.5275542262943093, None, 3]]
[[0.9910760598382222, 1.0, 0.231, 7], [0.9910760598382222, 0.7119046340726976, None, 3]]


In [146]:
print(C45)

{'密度': {'小于0.3815': {'label': 0, 'sampleNum': 4}, '大于0.3815': {'密度': {'小于0.5745': {'label': 1, 'sampleNum': 4}, '大于0.5745': {'含糖率': {'小于0.231': {'label': 0, 'sampleNum': 5}, '大于0.231': {'label': 1, 'sampleNum': 4}}}}}}}
